# Grammatical Error Correction with OpenVINO

AI-based auto-correction products are becoming increasingly popular due to their ease of use, editing speed, and affordability. These products improve the quality of written text in emails, blogs, and chats.

Grammatical Error Correction (GEC) is the task of correcting different types of errors in text such as spelling, punctuation, grammatical and word choice errors.
GEC is typically formulated as a sentence correction task. A GEC system takes a potentially erroneous sentence as input and is expected to transform it into a more correct version. See the example given below:

| Input (Erroneous)                                         | Output (Corrected)                                       |
| --------------------------------------------------------- | -------------------------------------------------------- |
| I like to rides my bicycle. | I like to ride my bicycle. |

 As shown in the image below, different types of errors in written language can be corrected.

![error_types](https://cdn-images-1.medium.com/max/540/1*Voez5hEn5MU8Knde3fIZfw.png)

This tutorial shows how to perform grammatical error correction using OpenVINO. We will use pre-trained models from the [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) library. To simplify the user experience, the [Hugging Face Optimum](https://huggingface.co/docs/optimum) library is used to convert the models to OpenVINO™ IR format.

It consists of the following steps:

- Install prerequisites
- Download and convert models from a public source using the [OpenVINO integration with Hugging Face Optimum](https://huggingface.co/blog/openvino).
- Create an inference pipeline for grammatical error checking
- Optimize grammar correction pipeline with [NNCF](https://github.com/openvinotoolkit/nncf/) quantization
- Compare original and optimized pipelines from performance and accuracy standpoints

#### Table of contents:
- [How does it work?](#How-does-it-work?-$\Uparrow$)
- [Prerequisites](#Prerequisites-$\Uparrow$)
- [Download and Convert Models](#Download-and-Convert-Models-$\Uparrow$)
    - [Select inference device](#Select-inference-device-$\Uparrow$)
    - [Grammar Checker](#Grammar-Checker-$\Uparrow$)
    - [Grammar Corrector](#Grammar-Corrector-$\Uparrow$)
- [Prepare Demo Pipeline](#Prepare-Demo-Pipeline-$\Uparrow$)
- [Interactive demo](#interactive-demo-$\Uparrow$)
- [Quantization](#quantization-$\Uparrow$)
    - [Prepare Calibration Dataset](#prepare-calibration-dataset-$\Uparrow$)
    - [Run Quantization](#run-quantization-$\Uparrow$)
    - [Run grammar correction with quantized OpenVINO model](#run-grammar-correction-with-quantized-openvino-model-$\Uparrow$)
    - [Compare model size, performance and accuracy](#compare-model-size-performance-and-accuracy-$\Uparrow$)
- [Interactive demo for quantized model](#interactive-demo-for-quantized-model-$\Uparrow$)


## How does it work? [$\Uparrow$](#Table-of-contents:)

A Grammatical Error Correction task can be thought of as a sequence-to-sequence task where a model is trained to take a grammatically incorrect sentence as input and return a grammatically correct sentence as output. We will use the [FLAN-T5](https://huggingface.co/pszemraj/flan-t5-large-grammar-synthesis) model finetuned on an expanded version of the [JFLEG](https://paperswithcode.com/dataset/jfleg) dataset.

The version of FLAN-T5 released with the [Scaling Instruction-Finetuned Language Models](https://arxiv.org/pdf/2210.11416.pdf) paper is an enhanced version of [T5](https://huggingface.co/t5-large) that has been finetuned on a combination of tasks. The paper explores instruction finetuning with a particular focus on scaling the number of tasks, scaling the model size, and finetuning on chain-of-thought data. The paper discovers that overall instruction finetuning is a general method that improves the performance and usability of pre-trained language models.

![flan-t5_training](https://production-media.paperswithcode.com/methods/a04cb14e-e6b8-449e-9487-bc4262911d74.png)

For more details about the model, please check out [paper](https://arxiv.org/abs/2210.11416), original [repository](https://github.com/google-research/t5x), and Hugging Face [model card](https://huggingface.co/google/flan-t5-large)

Additionally, to reduce the number of sentences required to be processed, you can perform grammatical correctness checking. This task should be considered as a simple binary text classification, where the model gets input text and predicts label 1 if a text contains any grammatical errors and 0 if it does not. You will use the [roberta-base-CoLA](https://huggingface.co/textattack/roberta-base-CoLA) model, the RoBERTa Base model finetuned on the CoLA dataset. The RoBERTa model was proposed in [RoBERTa: A Robustly Optimized BERT Pretraining Approach paper](https://arxiv.org/abs/1907.11692). It builds on BERT and modifies key hyperparameters, removing the next-sentence pre-training objective and training with much larger mini-batches and learning rates. Additional details about the model can be found in a [blog post](https://ai.facebook.com/blog/roberta-an-optimized-method-for-pretraining-self-supervised-nlp-systems/) by Meta AI and in the [Hugging Face documentation](https://huggingface.co/docs/transformers/model_doc/roberta)

Now that we know more about FLAN-T5 and RoBERTa, let us get started. 🚀

## Prerequisites [$\Uparrow$](#Table-of-contents:)

First, we need to install the [Hugging Face Optimum](https://huggingface.co/docs/transformers/index) library accelerated by OpenVINO integration.
The Hugging Face Optimum API is a high-level API that enables us to convert and quantize models from the Hugging Face Transformers library to the OpenVINO™ IR format. For more details, refer to the [Hugging Face Optimum documentation](https://huggingface.co/docs/optimum/intel/inference).

In [1]:
%pip install -q "git+https://github.com/huggingface/optimum-intel.git" "openvino>=2023.1.0" onnx onnxruntime gradio
%pip install -q "git+https://github.com/openvinotoolkit/nncf.git@9c671f0ae0a118e4bc2de8b09e66425931c0bfa4" datasets jiwer

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Download and Convert Models [$\Uparrow$](#Table-of-contents:)

Optimum Intel can be used to load optimized models from the [Hugging Face Hub](https://huggingface.co/docs/optimum/intel/hf.co/models) and create pipelines to run an inference with OpenVINO Runtime using Hugging Face APIs. The Optimum Inference models are API compatible with Hugging Face Transformers models.  This means we just need to replace `AutoModelForXxx` class with the corresponding `OVModelForXxx` class.

Below is an example of the RoBERTa text classification model

```diff
-from transformers import AutoModelForSequenceClassification
+from optimum.intel.openvino import OVModelForSequenceClassification
from transformers import AutoTokenizer, pipeline

model_id = "textattack/roberta-base-CoLA"
-model = AutoModelForSequenceClassification.from_pretrained(model_id)
+model = OVModelForSequenceClassification.from_pretrained(model_id, from_transformers=True)
```

Model class initialization starts with calling `from_pretrained` method. When downloading and converting Transformers model, the parameter `from_transformers=True` should be added. We can save the converted model for the next usage with the `save_pretrained` method.
Tokenizer class and pipelines API are compatible with Optimum models.

In [2]:
from pathlib import Path
from transformers import pipeline, AutoTokenizer
from optimum.intel.openvino import OVModelForSeq2SeqLM, OVModelForSequenceClassification

2023-09-22 19:33:42.740057: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-22 19:33:42.774517: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 19:33:43.312582: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda-11.7'
/home/nsavel/venvs/ov_notebooks_tmp/lib/python3.8/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


### Select inference device [$\Uparrow$](#Table-of-contents:)

select device from dropdown list for running inference using OpenVINO

In [3]:
import ipywidgets as widgets
import openvino as ov

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

### Grammar Checker [$\Uparrow$](#Table-of-contents:)


In [4]:
grammar_checker_model_id = "textattack/roberta-base-CoLA"
grammar_checker_dir = Path("roberta-base-cola")
grammar_checker_tokenizer = AutoTokenizer.from_pretrained(grammar_checker_model_id)

if grammar_checker_dir.exists():
    grammar_checker_model = OVModelForSequenceClassification.from_pretrained(grammar_checker_dir, device=device.value)
else:
    grammar_checker_model = OVModelForSequenceClassification.from_pretrained(grammar_checker_model_id, export=True, device=device.value)
    grammar_checker_model.save_pretrained(grammar_checker_dir)

Framework not specified. Using pt to export to ONNX.
Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using framework PyTorch: 1.13.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> False


[ WARNING ]  Please fix your imports. Module %s has been moved to %s. The old module will be deleted in version %s.
Compiling the model...
Set CACHE_DIR to /tmp/tmp_omqg7y6/model_cache


Let us check model work, using inference pipeline for `text-classification` task. You can find more information about usage Hugging Face inference pipelines in this [tutorial](https://huggingface.co/docs/transformers/pipeline_tutorial)

In [5]:
input_text = "They are moved by salar energy"
grammar_checker_pipe = pipeline("text-classification", model=grammar_checker_model, tokenizer=grammar_checker_tokenizer)
result = grammar_checker_pipe(input_text)[0]
print(f"input text: {input_text}")
print(f'predicted label: {"contains_errors" if result["label"] == "LABEL_1" else "no errors"}')
print(f'predicted score: {result["score"] :.2}')

input text: They are moved by salar energy
predicted label: contains_errors
predicted score: 0.88


Great! Looks like the model can detect errors in the sample.

### Grammar Corrector [$\Uparrow$](#Table-of-contents:)

The steps for loading the Grammar Corrector model are very similar, except for the model class that is used. Because FLAN-T5 is a sequence-to-sequence text generation model, we should use the `OVModelForSeq2SeqLM` class and the `text2text-generation` pipeline to run it.

In [6]:
grammar_corrector_model_id = "pszemraj/flan-t5-large-grammar-synthesis"
grammar_corrector_dir = Path("flan-t5-large-grammar-synthesis")
grammar_corrector_tokenizer = AutoTokenizer.from_pretrained(grammar_corrector_model_id)

if grammar_corrector_dir.exists():
    grammar_corrector_model = OVModelForSeq2SeqLM.from_pretrained(grammar_corrector_dir, device=device.value)
else:
    grammar_corrector_model = OVModelForSeq2SeqLM.from_pretrained(grammar_corrector_model_id, export=True, device=device.value)
    grammar_corrector_model.save_pretrained(grammar_corrector_dir)

Framework not specified. Using pt to export to ONNX.
Using framework PyTorch: 1.13.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> False
Using framework PyTorch: 1.13.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
/home/nsavel/venvs/ov_notebooks_tmp/lib/python3.8/site-packages/transformers/modeling_utils.py:875: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:
Using framework PyTorch: 1.13.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
/home/nsavel/venvs/ov_notebooks_tmp/lib/python3.8/site-packages/transformers/models/t5/modeling_t5.py:509: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so 

In [7]:
grammar_corrector_pipe = pipeline("text2text-generation", model=grammar_corrector_model, tokenizer=grammar_corrector_tokenizer)

In [8]:
result = grammar_corrector_pipe(input_text)[0]
print(f"input text:     {input_text}") 
print(f'generated text: {result["generated_text"]}') 

/home/nsavel/venvs/ov_notebooks_tmp/lib/python3.8/site-packages/optimum/intel/openvino/modeling_seq2seq.py:339: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  last_hidden_state = torch.from_numpy(self.request(inputs, shared_memory=True)["last_hidden_state"]).to(
/home/nsavel/venvs/ov_notebooks_tmp/lib/python3.8/site-packages/optimum/intel/openvino/modeling_seq2seq.py:416: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  self.request.start_async(inputs, shared_memory=True)


input text:     They are moved by salar energy
generated text: They are powered by solar energy.


Nice! The result looks pretty good!

## Prepare Demo Pipeline [$\Uparrow$](#Table-of-contents:)

Now let us put everything together and create the pipeline for grammar correction.
The pipeline accepts input text, verifies its correctness, and generates the correct version if required. It will consist of several steps:

1. Split text on sentences.
2. Check grammatical correctness for each sentence using Grammar Checker.
3. Generate an improved version of the sentence if required.

In [9]:
import re
import transformers
from tqdm.notebook import tqdm


def split_text(text: str) -> list:
    """
    Split a string of text into a list of sentence batches.

    Parameters:
    text (str): The text to be split into sentence batches.

    Returns:
    list: A list of sentence batches. Each sentence batch is a list of sentences.
    """
    # Split the text into sentences using regex
    sentences = re.split(r"(?<=[^A-Z].[.?]) +(?=[A-Z])", text)

    # Initialize a list to store the sentence batches
    sentence_batches = []

    # Initialize a temporary list to store the current batch of sentences
    temp_batch = []

    # Iterate through the sentences
    for sentence in sentences:
        # Add the sentence to the temporary batch
        temp_batch.append(sentence)

        # If the length of the temporary batch is between 2 and 3 sentences, or if it is the last batch, add it to the list of sentence batches
        if len(temp_batch) >= 2 and len(temp_batch) <= 3 or sentence == sentences[-1]:
            sentence_batches.append(temp_batch)
            temp_batch = []

    return sentence_batches


def correct_text(text: str, checker: transformers.pipelines.Pipeline, corrector: transformers.pipelines.Pipeline, separator: str = " ", disable_tqdm: bool = False) -> str:
    """
    Correct the grammar in a string of text using a text-classification and text-generation pipeline.

    Parameters:
    text (str): The inpur text to be corrected.
    checker (transformers.pipelines.Pipeline): The text-classification pipeline to use for checking the grammar quality of the text.
    corrector (transformers.pipelines.Pipeline): The text-generation pipeline to use for correcting the text.
    separator (str, optional): The separator to use when joining the corrected text into a single string. Default is a space character.

    Returns:
    str: The corrected text.
    """
    # Split the text into sentence batches
    sentence_batches = split_text(text)

    # Initialize a list to store the corrected text
    corrected_text = []

    # Iterate through the sentence batches
    for batch in tqdm(
        sentence_batches, total=len(sentence_batches), desc="correcting text..", disable=disable_tqdm
    ):
        # Join the sentences in the batch into a single string
        raw_text = " ".join(batch)

        # Check the grammar quality of the text using the text-classification pipeline
        results = checker(raw_text)

        # Only correct the text if the results of the text-classification are not LABEL_1 or are LABEL_1 with a score below 0.9
        if results[0]["label"] != "LABEL_1" or (
            results[0]["label"] == "LABEL_1" and results[0]["score"] < 0.9
        ):
            # Correct the text using the text-generation pipeline
            corrected_batch = corrector(raw_text)
            corrected_text.append(corrected_batch[0]["generated_text"])
        else:
            corrected_text.append(raw_text)

    # Join the corrected text into a single string
    corrected_text = separator.join(corrected_text)

    return corrected_text

Let us see it in action.

In [10]:
default_text = (
    "Most of the course is about semantic or  content of language but there are also interesting"
    " topics to be learned from the servicefeatures except statistics in characters in documents.At"
    " this point, He introduces herself as his native English speaker and goes on to say that if"
    " you contine to work on social scnce"
)

corrected_text = correct_text(default_text, grammar_checker_pipe, grammar_corrector_pipe)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


correcting text..:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
print(f"input text:     {default_text}\n") 
print(f'generated text: {corrected_text}')

input text:     Most of the course is about semantic or  content of language but there are also interesting topics to be learned from the servicefeatures except statistics in characters in documents.At this point, He introduces herself as his native English speaker and goes on to say that if you contine to work on social scnce

generated text: Most of the course is about the semantic content of language but there are also interesting topics to be learned from the service features except statistics in characters in documents. At this point, she introduces herself as a native English speaker and goes on to say that if you continue to work on social science, you will continue to be successful.


## Interactive demo [$\Uparrow$](#Table-of-contents:)

You can try out interactive demo right away or quantize the model first.

In [12]:
to_quantize = widgets.Dropdown(
    options=['Enable', 'Disable'],
    value='Disable',
    description='Quantization:',
    disabled=False,
)

to_quantize

Dropdown(description='Quantization:', options=('Enable', 'Disable'), value='Enable')

In [13]:
import gradio as gr


def correct(text, _=gr.Progress(track_tqdm=True)):
    return correct_text(text, grammar_checker_pipe, grammar_corrector_pipe)


demo = gr.Interface(
    correct,
    gr.Textbox(label="Text"),
    gr.Textbox(label="Correction"),
    examples=[default_text],
    allow_flagging="never",
)

if to_quantize.value == "Disable":
    # if you are launching remotely, specify server_name and server_port
    # demo.launch(server_name='your server name', server_port='server port in int')
    # Read more in the docs: https://gradio.app/docs/
    try:
        demo.queue().launch(debug=True)
    except Exception:
        demo.queue().launch(share=True, debug=True)
else:
    print("Skipping interactive demo for now")

Skipping interactive demo for now


## Quantization [$\Uparrow$](#Table-of-contents:)

[NNCF](https://github.com/openvinotoolkit/nncf/) enables post-training quantization by adding quantization layers into model graph and then using a subset of the training dataset to initialize the parameters of these additional quantization layers. Quantized operations are executed in `INT8` instead of `FP32`/`FP16` making model inference faster.

Grammar checker model takes up a tiny portion of the whole text correction pipeline so we optimize only the grammar corrector model. Grammar corrector itself consists of three models: encoder, first call decoder and decoder with past. The last model's share of inference time is about 90%. Because of this we quantize only it.

The optimization process contains the following steps:

1. Create a calibration dataset for quantization.
2. Run `nncf.quantize()` to obtain quantized models.
3. Serialize the `INT8` model using `openvino.runtime.serialize()` function.

### Prepare Calibration Dataset [$\Uparrow$](#Table-of-contents:)

In [14]:
import datasets
from contextlib import contextmanager

CALIBRATION_DATASET_SIZE = 10
COLLECT_CALIBRATION_DATA = False
calibration_data = []

@contextmanager
def calibration_data_collection():
    global COLLECT_CALIBRATION_DATA
    try:
        COLLECT_CALIBRATION_DATA = True
        yield
    finally:
        COLLECT_CALIBRATION_DATA = False

def wrap_for_data_collection(ov_decoder):
    original_fn = ov_decoder.request.start_async
    def wrapper(*args, **kwargs):
        inputs = kwargs.get("inputs", args[0])
        if COLLECT_CALIBRATION_DATA:
            calibration_data.append(inputs)
        return original_fn(*args, **kwargs)
    ov_decoder.request.start_async = wrapper


grammar_corrector_pipe_fp32 = grammar_corrector_pipe
ov_decoder = grammar_corrector_pipe_fp32.model.decoder_with_past

# Wrap decoder inference for data collection
wrap_for_data_collection(ov_decoder)

# Run inference for data collection
calibration_dataset = datasets.load_dataset("jfleg", split=f"validation[:{CALIBRATION_DATASET_SIZE}]").shuffle(seed=42)
with calibration_data_collection():
    for data_item in tqdm(calibration_dataset, total=CALIBRATION_DATASET_SIZE, desc="Collecting calibration data"):
        grammar_corrector_pipe_fp32(data_item["sentence"])

/home/nsavel/venvs/ov_notebooks_tmp/lib/python3.8/site-packages/optimum/intel/openvino/modeling_seq2seq.py:339: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  last_hidden_state = torch.from_numpy(self.request(inputs, shared_memory=True)["last_hidden_state"]).to(
/home/nsavel/venvs/ov_notebooks_tmp/lib/python3.8/site-packages/optimum/intel/openvino/modeling_seq2seq.py:416: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  self.request.start_async(inputs, shared_memory=True)
/tmp/ipykernel_83871/2962525502.py:24: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  return origin

### Run Quantization [$\Uparrow$](#Table-of-contents:)

Below we execute `nncf.quantize()` and save the quantized model.

In [15]:
import openvino.runtime as ov
import nncf
from nncf.quantization.range_estimator import RangeEstimatorParameters, StatisticsCollectorParameters, StatisticsType

quantized_model_path = Path("quantized_decoder_with_past") / "openvino_model.xml"


quantized_model = nncf.quantize(
    ov_decoder.model,
    calibration_dataset=nncf.Dataset(calibration_data),
    subset_size=len(calibration_data),
    model_type=nncf.ModelType.TRANSFORMER,
    advanced_parameters=nncf.AdvancedQuantizationParameters(
        disable_bias_correction=True,  # Disable bias correction because the model does not contain quantizable operations with bias
        smooth_quant_alpha=0.95,  # The value of 0.95 was selected by grid search
        activations_range_estimator_params=RangeEstimatorParameters(
            # Quantile statistic is employed due to outliers in some activations; this parameter was found useful by quantize_with_accuracy_control method
            max=StatisticsCollectorParameters(StatisticsType.QUANTILE)
        )
    ),
)

if not quantized_model_path.parent.exists():
    quantized_model_path.parent.mkdir(parents=True)
ov.serialize(quantized_model, quantized_model_path)

Output()

Output()

Output()

### Run grammar correction with quantized OpenVINO model [$\Uparrow$](#Table-of-contents:)

Create quantized grammar corrector pipeline and run correction based on it.

In [16]:
grammar_corrector_model_int8 = OVModelForSeq2SeqLM.from_pretrained(grammar_corrector_dir, device=device.value)
grammar_corrector_model_int8.decoder_with_past.model = quantized_model
grammar_corrector_model_int8.decoder_with_past.request = None
grammar_corrector_model_int8.decoder_with_past._compile()
grammar_corrector_pipe_int8 = pipeline("text2text-generation", model=grammar_corrector_model_int8, tokenizer=grammar_corrector_tokenizer)

default_text = (
    "Most of the course is about semantic or  content of language but there are also interesting"
    " topics to be learned from the servicefeatures except statistics in characters in documents.At"
    " this point, He introduces herself as his native English speaker and goes on to say that if"
    " you contine to work on social scnce"
)

corrected_text_int8 = correct_text(default_text, grammar_checker_pipe, grammar_corrector_pipe_int8)

# Let's see the results. The generated texts for quantized INT8 model and original FP32 model should be almost the same.
print(f"Input text:                   {default_text}\n")
print(f'Generated text by INT8 model: {corrected_text_int8}')

Compiling the encoder...
Compiling the decoder...
Compiling the decoder...
Compiling the decoder...


correcting text..:   0%|          | 0/1 [00:00<?, ?it/s]

Input text:                   Most of the course is about semantic or  content of language but there are also interesting topics to be learned from the servicefeatures except statistics in characters in documents.At this point, He introduces herself as his native English speaker and goes on to say that if you contine to work on social scnce

Generated text by INT8 model: Most of the course is about the semantic content of language but there are also interesting topics to be learned from the service features except statistics in characters in documents. At this point, she introduces herself as a native English speaker and goes on to say that if you continue to work on social science, you will continue to be successful.


### Compare model size, performance and accuracy [$\Uparrow$](#Table-of-contents:)

First, we compare file size of `FP32` and `INT8` models.

In [17]:
def calculate_compression_rate(model_path_ov, model_path_ov_int8):
    model_size_fp32 = model_path_ov.with_suffix(".bin").stat().st_size / 1024
    model_size_int8 = model_path_ov_int8.with_suffix(".bin").stat().st_size / 1024
    print(f"Model: {model_path_ov.stem}")
    print(f"    * FP32 IR model size: {model_size_fp32:.2f} KB")
    print(f"    * INT8 IR model size: {model_size_int8:.2f} KB")
    print(f"    * Model compression rate: {model_size_fp32 / model_size_int8:.3f}")

calculate_compression_rate(grammar_corrector_dir / "openvino_decoder_with_past_model.xml", quantized_model_path)

Model: openvino_decoder_with_past_model
    * FP32 IR model size: 1658150.26 KB
    * INT8 IR model size: 416958.33 KB
    * Model compression rate: 3.977


Second, we compare two grammar correction pipelines from performance and accuracy stand points.

We again use the same <spell>[jfleg](https://huggingface.co/datasets/jfleg)<spell> dataset, but in this case the test split it selected. One dataset sample consists of a text with errors as input and several corrected versions as labels.

When measuring accuracy we use mean `(1 - WER)` against corrected text versions, where WER is Word Error Rate metric.

In [18]:
import time
from jiwer import wer, wer_standardize

TEST_DATASET_SIZE = 50
test_dataset = datasets.load_dataset("jfleg", split=f"test[:{TEST_DATASET_SIZE}]")

def calculate_inference_time_and_accuracy(grammar_corrector_pipe):
    ground_truths = []
    predictions = []
    inference_time = []
    for data_item in tqdm(test_dataset, total=TEST_DATASET_SIZE, desc="Evaluation"):
        input_text = data_item["sentence"]  # e.g., "For not use car . "
        references = data_item["corrections"]  # e.g., [ "Not for use with a car . ", "Do not use in the car . ", "Car not for use . ", "Can not use the car . " ]

        start_time = time.perf_counter()
        corrected_text = correct_text(input_text, grammar_checker_pipe, grammar_corrector_pipe, disable_tqdm=True)
        end_time = time.perf_counter()
        delta_time = end_time - start_time

        ground_truths.extend(references)
        predictions.extend([corrected_text] * len(references))
        inference_time.append(delta_time)

    word_accuracy = (1 - wer(ground_truths, predictions, reference_transform=wer_standardize, hypothesis_transform=wer_standardize)) * 100
    sum_inference_time = sum(inference_time)
    return sum_inference_time, word_accuracy

inference_time_fp32, accuracy_fp32 = calculate_inference_time_and_accuracy(grammar_corrector_pipe_fp32)
print(f"Evaluation results of FP32 grammar correction pipeline. Accuracy: {accuracy_fp32:.2f}%. Time: {inference_time_fp32:.2f} sec.")
inference_time_int8, accuracy_int8 = calculate_inference_time_and_accuracy(grammar_corrector_pipe_int8)
print(f"Evaluation results of INT8 grammar correction pipeline. Accuracy: {accuracy_int8:.2f}%. Time: {inference_time_int8:.2f} sec.")
print(f"Performance speedup: {inference_time_fp32 / inference_time_int8:.3f}")
print(f"Accuracy drop :{accuracy_fp32 - accuracy_int8:.2f}%.")

Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

/tmp/ipykernel_83871/2962525502.py:24: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  return original_fn(*args, **kwargs)


Evaluation results of FP32 grammar correction pipeline. Accuracy: 67.59%. Time: 53.11 sec.


Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluation results of INT8 grammar correction pipeline. Accuracy: 67.92%. Time: 37.58 sec.
Performance speedup: 1.413
Accuracy drop :-0.33%.


## Interactive demo for quantized model [$\Uparrow$](#Table-of-contents:)

In [ ]:
import gradio as gr


def correct(text, quantized, progress=gr.Progress(track_tqdm=True)):
    grammar_corrector = grammar_corrector_pipe_int8 if quantized else grammar_corrector_pipe_fp32
    
    start_time = time.perf_counter()
    corrected_text = correct_text(text, grammar_checker_pipe, grammar_corrector)
    end_time = time.perf_counter()
    
    return corrected_text, f"{end_time - start_time:.2f}"
    


demo = gr.Interface(
    fn=correct,
    inputs=[gr.Textbox(label="Text"), gr.Checkbox(True, label="Quantized")],
    outputs=[gr.Textbox(label="Correction"), gr.Textbox(label="Time (seconds)")],
    examples=[[default_text, True]],
    allow_flagging="never",
)

# if you are launching remotely, specify server_name and server_port
# demo.launch(server_name='your server name', server_port='server port in int')
# Read more in the docs: https://gradio.app/docs/
try:
    demo.queue().launch(debug=True)
except Exception:
    demo.queue().launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


correcting text..:   0%|          | 0/1 [00:00<?, ?steps/s]

/home/nsavel/venvs/ov_notebooks_tmp/lib/python3.8/site-packages/optimum/intel/openvino/modeling_seq2seq.py:339: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  last_hidden_state = torch.from_numpy(self.request(inputs, shared_memory=True)["last_hidden_state"]).to(
/home/nsavel/venvs/ov_notebooks_tmp/lib/python3.8/site-packages/optimum/intel/openvino/modeling_seq2seq.py:416: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  self.request.start_async(inputs, shared_memory=True)


correcting text..:   0%|          | 0/1 [00:00<?, ?steps/s]

/tmp/ipykernel_83871/2962525502.py:24: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  return original_fn(*args, **kwargs)


correcting text..:   0%|          | 0/1 [00:00<?, ?steps/s]